In [3]:
!git clone https://github.com/soCzech/TransNetV2.git

fatal: destination path 'TransNetV2' already exists and is not an empty directory.


In [1]:
%cd ../support_models/TransNetV2/

/home/students/ttnhan-cse23/hcmai/support_models/TransNetV2


In [2]:
!git lfs fetch https://github.com/soCzech/TransNetV2.git

fetch: Fetching reference refs/heads/master


In [3]:
!git lfs checkout

Checking out LFS objects: 100% (3/3), 36 MB | 0 B/s, done.                      


In [6]:
%cd ../../notebook

/home/students/ttnhan-cse23/hcmai/notebook


In [ ]:
%pip install tensorflow=2.1 opencv-python matplotlib ffmpeg-python

In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [6]:
import os
import sys
import cv2
import imageio
from tqdm import tqdm
import json

In [3]:
import sys
sys.path.append("/home/students/ttnhan-cse23/hcmai")

### Create a Dictionary to store video path

In [15]:
import os
import imageio

dataset_dir = "/home/students/ttnhan-cse23/hcmai/dataset"

all_video_paths = dict()

for folder in sorted(os.listdir(dataset_dir)):
    if folder == "extracted_frames" or folder == "extracted_keyframes_JSON":
        continue
    all_video_paths[folder] = dict()
print(all_video_paths)

for folder in sorted(all_video_paths.keys()):
    for video in sorted(os.listdir(os.path.join(dataset_dir, folder))):
        video_id = video.replace(".mp4", "").split("_")[-1]
        video_path = f"{dataset_dir}/{folder}/{video}"
        all_video_paths[folder][video_id] = video_path
print(all_video_paths)

{'L01': {}}
{'L01': {'V001': '/home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V001.mp4', 'V002': '/home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V002.mp4', 'V003': '/home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V003.mp4', 'V004': '/home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V004.mp4', 'V005': '/home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V005.mp4'}}


### TransNetV2

In [ ]:
from tqdm import tqdm
import json

from support_models.TransNetV2.inference.transnetv2 import TransNetV2
model = TransNetV2()

json_save_dir = "/home/students/ttnhan-cse23/hcmai/dataset/extracted_keyframes_JSON"

for folder_name, video_paths_dict  in tqdm(all_video_paths.items()):                    # using tqdm for progress bar display
    videos_id = sorted(video_paths_dict.keys())
    
    if not os.path.exists(os.path.join(json_save_dir, folder_name)):
        os.makedirs(os.path.join(json_save_dir, folder_name))
    
    for video_id in tqdm(videos_id):
        video_path = video_paths_dict[video_id]
        
        # Scene Change Detection
        _, single_frame_prediction, _ = model.predict_video(video_path)
        list_of_scenes = model.predictions_to_scenes(predictions=single_frame_prediction)
        
        with open(f"{json_save_dir}/{folder_name}/{video_id}.json", "w") as f:
            json.dump(list_of_scenes.tolist(), f)

[TransNetV2] Using weights from /home/students/ttnhan-cse23/hcmai/support_models/TransNetV2/inference/transnetv2-weights/.


2025-03-12 01:39:31.468512: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2025-03-12 01:39:31.472301: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2995200000 Hz
2025-03-12 01:39:31.472927: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5611c3e4a360 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-03-12 01:39:31.472934: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-03-12 01:39:31.472982: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
  0%|          | 0/2 [00:00<?, ?it/s]

[TransNetV2] Extracting frames from /home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V001.mp4
[TransNetV2] Processing video frames 31665/31665


[TransNetV2] Extracting frames from /home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V002.mp4
[TransNetV2] Processing video frames 24300/24337

[TransNetV2] Processing video frames 24337/24337
[TransNetV2] Extracting frames from /home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V003.mp4
[TransNetV2] Processing video frames 30668/30668


[TransNetV2] Extracting frames from /home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V004.mp4
[TransNetV2] Processing video frames 23000/23027

[TransNetV2] Processing video frames 23027/23027
[TransNetV2] Extracting frames from /home/students/ttnhan-cse23/hcmai/dataset/L01/L01_V005.mp4
[TransNetV2] Processing video frames 29000/29024

 50%|█████     | 1/2 [09:34<09:34, 574.36s/it]

[TransNetV2] Processing video frames 29024/29024


0it [00:00, ?it/s]
100%|██████████| 2/2 [09:34<00:00, 287.18s/it]


In [1]:
def sample_frames_from_shot(start_idx, end_idx):
    '''
    intervals = np.linspace(start=start_idx, stop=end_idx, num=n_frames+1).astype(int)
    ranges = []
    for idx, interv in enumerate(intervals[:-1]):
        ranges.append((interv, intervals[idx + 1]))
    frame_idxs = [(x[0] + x[1]) // 2 for x in ranges]
    '''
    middle_idx = (start_idx + end_idx) // 2
    frame_idxs = [start_idx, middle_idx, end_idx]
    return frame_idxs

In [13]:
import cv2

scene_json_dir = "/home/students/ttnhan-cse23/hcmai/dataset/extracted_keyframes_JSON"
extracted_keyframes_dir = "/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames"

for key in all_video_paths.keys():
    save_dir = os.path.join(extracted_keyframes_dir, key)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    video_paths_dict = all_video_paths[key]
    video_ids = sorted(video_paths_dict.keys())
    
    for video_id in tqdm(video_ids):
        video_path = video_paths_dict[video_id]
        video_scene_path = f"{scene_json_dir}/{key}/{video_id}.json"
        
        with open(video_scene_path, "r") as f:
            video_scenes = json.load(f)
        
        if not os.path.exists(os.path.join(save_dir, video_id)):
            os.mkdir(os.path.join(save_dir, video_id))
            
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Error: Unable to open video {video_path}")
            continue
        for i, shot in enumerate(tqdm(video_scenes)):
            shot_frames_id = sample_frames_from_shot(shot[0], shot[1])
            for index in shot_frames_id:
                cap.set(cv2.CAP_PROP_POS_FRAMES, index)
                filename = "{}/{:0>6d}.jpg".format(f"{save_dir}/{video_id}", index)
                ret, frame = cap.read()
                if ret:
                    if not cv2.imwrite(filename, frame):
                        print("failed save")
                    else:
                        pass
        cap.release()

  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/students/ttnhan-cse23/hcmai/dataset/extracted_keyframes_JSON/extracted_keyframes_JSON/L01.json'